# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.47.0


## Dataset


### Overview
The dataset that will be used in this experiment is from Kaggle. It is a tablular dataset that contains features related to polished diamonds such as the cut, carat weight and colour. The aim of the data is to use these features to predict the price of the polished diamond.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

In [4]:
# choose a name for experiment
experiment_name = 'DiamondPriceAutoML'

experiment=Experiment(ws, experiment_name)

found = False
key = "diamond-data"
description_text = "Dataset to predict the price of round cut polished diamonds"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("Dataset not found, please register dataset in the workspace")


df = dataset.to_pandas_dataframe()
df.describe()

,carat,depth,table,price,x,y,z
count,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000,53943.000000
mean,0.797935,61.749322,57.457251,3932.734294,5.731158,5.734526,3.538730
std,0.473999,1.432626,2.234549,3989.338447,1.121730,1.142103,0.705679
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


Split the data into a training set and a test set that we can use to evaluate the model


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.2, random_state=223)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The AutoML task is set to regression as we want to predict a numerical value (diamond price) from a set of features. We will use the compute cluster that already exists in the workspace to train the model. The experiment is set to time out after 30 minutes so that we don't consume an indefinite amount of resources. The primary metric that will be optimised is normalized_root_mean_squared_error as we want to minimise the average squared error. As the dataset is not very large, 5 cross-validations are used. Featurisation is enabled as no data cleaning was done after reading in the data and inputting it to the AutoML model. Early stopping is also enabled.


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
print('Found existing cluster, use it.')

Found existing cluster, use it.


In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.5,
    "enable_early_stopping": True,
    "compute_target" : compute_target,   
    "primary_metric": 'normalized_root_mean_squared_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=dataset,
                             label_column_name="price",
                             **automl_settings)

In [8]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

SDK version: 1.47.0
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
DiamondPriceAutoML,AutoML_823d2abe-040d-4067-ad93-a899f8cae559,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

A total of 27 models were trained using AutoML. Of these, the best performing model was a Voting Ensemble. This makes sense as it uses multiple models to make a decision. 

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run_automl, fitted_model = remote_run.get_output()
print('Best Run: ' + str(best_run_automl.id))
RunDetails(best_run_automl).show()
print(best_run_automl)
print(fitted_model)

Best Run: AutoML_823d2abe-040d-4067-ad93-a899f8cae559_30


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Run(Experiment: DiamondPriceAutoML,
Id: AutoML_823d2abe-040d-4067-ad93-a899f8cae559_30,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScale

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Test the best model


In [10]:
y_test = x_test.pop("price")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


[3100.62425414  703.11238131 3071.4771745  9004.56238141 3208.15516464
 2332.04973945 1349.16823929 9505.11320604 1723.5978327  1207.30644312]
Model MAPE:
0.06232559241567767

Model Accuracy:
0.9376744075843223


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [14]:
best_run = remote_run.get_best_child()
model_name = best_run.properties["model_name"]

script_file_name = "inference/score.py"

best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")
description = "AutoML Model trained on diamond data to predict the diamond price"
tags = None
model = remote_run.register_model(
    model_name=model_name, description=description, tags=tags
)

print(
    remote_run.model_id
)  # This will be written to the script file later in the notebook.

AutoML823d2abe030


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={"area": "diamondData", "type": "automl_classification"},
    description="Diamond data service for Automl Classification",
)

aci_service_name = model_name.lower()
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl823d2abe030
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-09 11:49:25+00:00 Creating Container Registry if not exists.
2022-11-09 11:49:26+00:00 Use the existing image.
2022-11-09 11:49:27+00:00 Submitting deployment to compute.
2022-11-09 11:49:30+00:00 Checking the status of deployment automl823d2abe030..
2022-11-09 11:51:43+00:00 Checking the status of inference endpoint automl823d2abe030.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
from numpy import array
import json


import requests
X_test = x_test.head(3)
print(X_test)
X_test_json = X_test.to_json(orient="records")
data = '{"data": ' + X_test_json + "}"
headers = {"Content-Type": "application/json"}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)

y_pred = json.loads(json.loads(resp.text))["result"]
print(y_pred)


      Column2  carat      cut color clarity  depth  table    x    y    z
1495     1496   0.70    Ideal     D     VS2  60.30  60.00 5.71 5.76 3.46
30881   30882   0.33    Ideal     E     SI1  62.00  54.00 4.43 4.41 2.74
53372   53373   0.71  Premium     D     VS2  62.70  58.00 5.53 5.67 3.54
[3100.6242541355014, 703.1123813054058, 3071.477174501366]


TODO: In the cell below, print the logs of the web service and delete the service

In [29]:
print(aci_service.get_logs())

2022-11-08T09:29:19,715462100+00:00 - rsyslog/run 
2022-11-08T09:29:19,715525400+00:00 - iot-server/run 
2022-11-08T09:29:19,718337000+00:00 - gunicorn/run 
2022-11-08T09:29:19,722549300+00:00 | gunicorn/run | 
2022-11-08T09:29:19,724357800+00:00 | gunicorn/run | ###############################################
2022-11-08T09:29:19,726091900+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-11-08T09:29:19,729009700+00:00 | gunicorn/run | ###############################################
2022-11-08T09:29:19,733395500+00:00 | gunicorn/run | 
2022-11-08T09:29:19,737055600+00:00 | gunicorn/run | 
2022-11-08T09:29:19,750494600+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220930.v4
2022-11-08T09:29:19,753002600+00:00 | gunicorn/run | 
2022-11-08T09:29:19,754634600+00:00 | gunicorn/run | 
2022-11-08T09:29:19,756822100+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_167ced737ee9811a93e21408b63f0ab3/bin:/o

# Delete the webservice and computes


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

aci_service.delete()
delete_compute(compute_target)

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
